In [5]:
import pandas as pd
import sqlite3
from sqlite3 import Error

In [4]:
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

import pandas as pd
df=pd.read_csv('AB_US_2020.csv')
df.to_csv('airbnb_new_data.tsv',sep='\t')


In [ ]:
'id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city'

## GET AIRBNB DATA INTO SQL DB without city data##

In [ ]:


def csv_to_db(file_name,table_name,db_name,create_sql_statement,columns):
    with open(file_name, encoding="mbcs") as fl:
        cnt=0
        data=[]
        for lines in fl:
            #print(lines)
            if cnt!=0:
                lst=lines.strip().split('\t')
                row_tmp = tuple(lst) 
                if len(row_tmp)==18:
                    data.append(row_tmp)
            cnt+=1
    ##print(data)
    conn = create_connection(db_name)
    cur = conn.cursor()
    
    ### CREATE TABLE ##
    create_table(conn,create_sql_statement,table_name)
    
    ### get number of columns
    col_num=len(columns.split(','))
    insert_sql='insert into '+str(table_name)+' values('
    for i in range(col_num):
        insert_sql+='?,'
    
    insert_sql=insert_sql[:-1]+')'
    print(insert_sql)
    
    
    cur.executemany(insert_sql,data)
    conn.commit()
    conn.close()
    
    return data


    
create_table_sql='''
 create table airbnb_data (
          row_id integer primary key not null,
          id integer,
          name text,
          host_id integer,
          host_name text,
          neighbourhood_group text,
          neighbourhood text,
          latitude real,
          longitude real,
          room_type text,
          price real,
          minimum_nights integer,
          number_of_reviews integer,
          last_review text,
          reviews_per_month real,
          calculated_host_listings_count integer,
          availability_365 integer,
          city text
        );
'''
data = csv_to_db('airbnb_new_data.tsv','airbnb_data','airbnb.db',create_table_sql,'row_id,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city')





## GET US CITY DATA INTO SQL DB ##


In [ ]:
def city_state_us(file_name,db_name,table_name,columns,create_table_sql):
    with open(file_name, encoding="mbcs") as fl:
            cnt=0
            data=[]
            for lines in fl:
                ##print(lines)
                if cnt!=0:
                    lst=lines.strip().split(',')
                    row_tmp = tuple(lst)
                    data.append(row_tmp)
                cnt+=1
        
        ##print(data)
        
    conn = create_connection(db_name)
    cur = conn.cursor()

    ### CREATE TABLE ##
    create_table(conn,create_table_sql,table_name)
                
    ### get number of columns
    col_num=len(columns.split(','))
    insert_sql='insert into '+str(table_name)+' values('
    for i in range(col_num):
        insert_sql+='?,'
    insert_sql=insert_sql[:-1]+')'
    print(insert_sql)
        
    ### INSERT INTO TABLE ###
    cur.executemany(insert_sql,data)
    conn.commit()
    conn.close()
    return data

create_table_sql_city_states='''
 create table city_state_mapping (
          ID integer not null primary key,
          STATE_CODE text,
          STATE_NAME text,
          CITY text,
          COUNTY text,
          LATITUDE real,
          LONGITUDE real
        );
'''
data1= city_state_us('us_cities.csv','airbnb.db','city_state_mapping','ID,STATE_CODE,STATE_NAME,CITY,COUNTY,LATITUDE,LONGITUDE',create_table_sql_city_states)

## CHECK  if cities in airbnb dataset have duplicate names present in city state dataset ##

In [9]:
def sql_query(sql_statement,db_name):
    conn = create_connection(db_name)
    cur = conn.cursor()
    df =pd.read_sql_query(sql_statement, conn)    
    conn.commit()
    conn.close()
    return df

sql=''' 
select a.city,count(*) from 
(select distinct case when substr(city,1,1)='"' then lower(trim(substr(city,2,length(city)-2))) else lower(trim(city)) end as city from city_state_mapping
) a
where lower(trim(city)) in (
select distinct case when substr(city,1,1)='"' then lower(trim(substr(city,2,length(city)-2))) else lower(trim(city)) end as city from airbnb_data
)
group by a.city;
          '''



sql_query(sql,'airbnb.db')

,city,count(*)
0,asheville,1
1,austin,1
2,boston,1
3,cambridge,1
4,chicago,1
5,columbus,1
6,denver,1
7,jersey city,1
8,los angeles,1
9,nashville,1


## Check the city distribution of AIRBNBs ##

In [29]:

sql=''' 
select a.city,count(*) from 
airbnb_data a
where lower(trim(city)) in (
select distinct case when substr(city,1,1)='"' then lower(trim(substr(city,2,length(city)-2))) else lower(trim(city)) end as city from airbnb_data
)
group by a.city
order by count(*) desc
'''
sql_query(sql,'airbnb.db')


,city,count(*)
0,New York City,45629
1,Los Angeles,31397
2,Hawaii,22385
3,San Diego,12360
4,Broward County,10815
5,Austin,10415
6,Clark County,8369
7,Washington D.C.,7343
8,San Clara Country,7078
9,San Francisco,7044


## CITY STATE MAPPING TABLE FORMATTED ##

In [6]:

def sql_query_create_table_as(sql_statement,db_name):
    conn = create_connection(db_name)
    cur = conn.cursor()
    cur.execute(sql)    
    conn.commit()
    conn.close()

sql='''
create table state_city_mapping_transformed as
select  id,state_code,case when substr(STATE_NAME,1,1)='"' then trim(substr(STATE_NAME,2,length(STATE_NAME)-2)) else trim(STATE_NAME) end as state_name,case when substr(COUNTY,1,1)='"' then trim(substr(COUNTY,2,length(COUNTY)-2)) else trim(COUNTY) end as county,LATITUDE,LONGITUDE,case when substr(city,1,1)='"' then trim(substr(city,2,length(city)-2)) else trim(city) end as city from city_state_mapping'''

sql_query_create_table_as(sql,'airbnb.db')


OperationalError: table state_city_mapping_transformed already exists

## Transformed CITY STATE dataset ##

In [ ]:
sql_query('select * from state_city_mapping_transformed','airbnb.db')

### API to get state names from GEOCODE ###

In [ ]:
!pip install geopy


In [13]:
from geopy.geocoders import Nominatim

# initialize Nominatim API 
geolocator = Nominatim(user_agent="geoapiExercises")
Latitude = "35.65146"
Longitude = "-82.62791999999999"
location = geolocator.reverse(Latitude+","+Longitude)
address=location.raw['address']  
# Display
print(address.get('state'))

North Carolina


## UPDATED INGESTION WITH STATE NAMES ##

In [24]:


def csv_to_db(file_name,table_name,db_name,create_sql_statement,columns):
    geolocator = Nominatim(user_agent="geoapiExercises")
    
    with open(file_name, encoding="mbcs") as fl:
        cnt=0
        data=[]
        for lines in fl:
            #print(lines)
            if cnt!=0:
                
                lst=lines.strip().split('\t')
                if len(lst)==18:
                    data.append(lst)
            cnt+=1
    ##print(data)
    conn = create_connection(db_name)
    cur = conn.cursor()
    
    ### CREATE TABLE ##
    create_table(conn,create_sql_statement,table_name)
    
    
    #### GET distinct city names ###
    city={}
    for i in data:
        if i[-1] not in city.keys():
            city[i[-1]]=(i[7],i[8])
    
    
    #### mapping latitudes to states ####
    
    for i in city.keys():
        Latitude = city[i][0]
        Longitude = city[i][1]
        print(Latitude,Longitude)
        location = geolocator.reverse(Latitude+","+Longitude)
        print(i,location)
        address=location.raw['address']
        state=address.get('state')
        
        city[i]=state
    
    #### ADD states to existing data ###
    
    final_data=[]
    for i in data:
        i.append(city[i[-1]])
        final_data.append(tuple(i))
    
    
    ### get number of columns
    col_num=len(columns.split(','))
    insert_sql='insert into '+str(table_name)+' values('
    for i in range(col_num):
        insert_sql+='?,'
    
    insert_sql=insert_sql[:-1]+')'
    print(insert_sql)
    
    
    cur.executemany(insert_sql,final_data)
    conn.commit()
    conn.close()
    
    return data,city


    
create_table_sql='''
 create table airbnb_data_new (
          row_id integer primary key not null,
          id integer,
          name text,
          host_id integer,
          host_name text,
          neighbourhood_group text,
          neighbourhood text,
          latitude real,
          longitude real,
          room_type text,
          price real,
          minimum_nights integer,
          number_of_reviews integer,
          last_review text,
          reviews_per_month real,
          calculated_host_listings_count integer,
          availability_365 integer,
          city text,
          state text
        );
'''
data,city = csv_to_db('airbnb_new_data.tsv','airbnb_data_new','airbnb.db',create_table_sql,'row_id,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,city,state')




35.65146 -82.62791999999999
Asheville 1189, Olivette Road, Woodfin, Buncombe County, North Carolina, 28804, United States
30.2775 -97.71398
Austin 2507, East 16th Street, Glenwood, Austin, Travis County, Texas, 78702, United States
42.364129999999996 -71.02991
Boston 50, Jeffries Street, Jeffries Point, East Boston, Boston, Suffolk County, Massachusetts, 02128-2909, United States
26.12814 -80.10259
Broward County North Fort Lauderdale Beach Boulevard, Birch Ocean Front, Fort Lauderdale, Broward County, Florida, 33305, United States
42.38329 -71.13616999999999
Cambridge 55, Granville Road, Huron Village, Cambridge, Middlesex County, Massachusetts, 02140, United States
41.7879 -87.5878
Chicago 1518-1534, East 59th Street, Hyde Park, Chicago, Hyde Park Township, Cook County, Illinois, 60637, United States
36.038540000000005 -115.12102
Clark County 8318, Vista Colorado Street, Paradise, Clark County, Nevada, 89123, United States
39.983940000000004 -83.00321
Columbus 60, East Third Avenue, 

In [22]:
city

{'Asheville': 'North Carolina',
 'Austin': 'Texas',
 'Boston': 'Massachusetts',
 'Broward County': 'Florida',
 'Cambridge': 'Massachusetts',
 'Chicago': 'Illinois',
 'Clark County': 'Nevada',
 'Columbus': 'Ohio',
 'Denver': 'Colorado',
 'Hawaii': 'Hawaii',
 'Jersey City': 'New Jersey',
 'Los Angeles': 'California',
 'Nashville': 'Tennessee',
 'New Orleans': 'Louisiana',
 'New York City': 'New York',
 'Oakland': 'California',
 'Pacific Grove': 'California',
 'Portland': 'Oregon',
 'Rhode Island': 'Rhode Island',
 'Salem': 'Oregon',
 'San Clara Country': 'California',
 'San Diego': 'California',
 'San Francisco': 'California',
 'San Mateo County': 'California',
 'Santa Cruz County': 'California',
 'Seattle': 'Washington',
 'Twin Cities MSA': 'Minnesota',
 'Washington D.C.': 'District of Columbia'}

In [28]:
sql=''' select state,count(*) from airbnb_data_new group by state order by count(*) desc '''
sql_query(sql,'airbnb.db')

,state,count(*)
0,California,65673
1,New York,45629
2,Hawaii,22385
3,Florida,10815
4,Texas,10415
5,Nevada,8369
6,District of Columbia,7343
7,Washington,6562
8,Minnesota,6460
9,Louisiana,6392


## NORMALIZE DATASET ##

## Other Datasets ingest ##

In [ ]:
b